# Dynamic Programming on FrozenLake (Policy Evaluation & Iteration)

Learn how to solve an MDP exactly using Dynamic Programming (DP):
- Policy Evaluation
- Policy Improvement
- Policy Iteration
- Value Iteration

Environment: `FrozenLake-v1` (map_size=8, is_slippery=True)


In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True)
S = env.observation_space.n
A = env.action_space.n
P = env.unwrapped.P  # transition dynamics dict
print("States:", S, "Actions:", A)


## Policy Evaluation
Given a policy π, compute V^π by iteratively applying the Bellman expectation backup until convergence.


In [ ]:
def policy_evaluation(policy, P, gamma=0.99, tol=1e-8):
    V = np.zeros(S)
    while True:
        delta = 0
        for s in range(S):
            v = 0
            a = policy[s]
            for prob, s2, r, done in P[s][a]:
                v += prob * (r + gamma * V[s2])
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < tol:
            break
    return V

# random policy to start
policy = np.random.randint(0, A, size=S)
V = policy_evaluation(policy, P)
V[:10]


## Policy Improvement
Given V^π, greedify the policy with respect to the action-value estimates.


In [ ]:
def policy_improvement(V, P, gamma=0.99):
    policy = np.zeros(S, dtype=int)
    for s in range(S):
        q = np.zeros(A)
        for a in range(A):
            for prob, s2, r, done in P[s][a]:
                q[a] += prob * (r + gamma * V[s2])
        policy[s] = int(np.argmax(q))
    return policy

new_policy = policy_improvement(V, P)
new_policy[:20]


## Policy Iteration
Alternate policy evaluation and improvement until the policy stabilizes.


In [ ]:
def policy_iteration(P, gamma=0.99):
    policy = np.random.randint(0, A, size=S)
    while True:
        V = policy_evaluation(policy, P, gamma)
        new_policy = policy_improvement(V, P, gamma)
        if np.all(policy == new_policy):
            return policy, V
        policy = new_policy

pi, V_pi = policy_iteration(P)
print("Policy iteration done. Sample V:", V_pi[:10])


## Value Iteration
Directly iterate on V using the Bellman optimality operator until convergence, then extract a greedy policy.


In [ ]:
def value_iteration(P, gamma=0.99, tol=1e-8):
    V = np.zeros(S)
    while True:
        delta = 0
        for s in range(S):
            q = np.zeros(A)
            for a in range(A):
                for prob, s2, r, done in P[s][a]:
                    q[a] += prob * (r + gamma * V[s2])
            v_new = np.max(q)
            delta = max(delta, abs(v_new - V[s]))
            V[s] = v_new
        if delta < tol:
            break
    # greedy policy
    pi = np.zeros(S, dtype=int)
    for s in range(S):
        q = np.zeros(A)
        for a in range(A):
            for prob, s2, r, done in P[s][a]:
                q[a] += prob * (r + gamma * V[s2])
        pi[s] = int(np.argmax(q))
    return pi, V

pi_opt, V_opt = value_iteration(P)
print("Value iteration done. Sample V:", V_opt[:10])
